In [1]:
import sys
sys.path.append('../')
from pathlib import Path
import requests
import os
if not os.path.isdir('./.data'):
    os.mkdir('./.data')
import json
import numpy as np
import pandas as pd
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
import matplotlib.pyplot as plt
%matplotlib inline

import time
import datetime
import pytz
import warnings; warnings.simplefilter('ignore')

In [2]:
# Sklearn stuff
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score, StratifiedKFold, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

SEED = 17

In [36]:
# Import Training and Test Data  
train_features = pd.read_csv('data2/dpc_train_features.csv', index_col='match_id')
train_targets = pd.read_csv('data2/dpc_train_target.csv', index_col='match_id')
test_features = pd.read_csv('data2/dpc_test_features.csv', index_col='match_id')
test_targets = pd.read_csv('data2/dpc_test_target.csv', index_col='match_id')

In [37]:
train_features.shape, train_targets.shape, test_features.shape, test_targets.shape

((1045, 66), (1045, 2), (117, 66), (117, 2))

In [54]:
# Label targets
y_train = train_targets.radiant_win.values
y_test = test_targets.radiant_win.values

## 

In [41]:
df_full_features = pd.concat([train_features, test_features])

# Index to split the training and test data sets
idx_split = train_features.shape[0]

heroes_df = df_full_features[[f'{i}hero_id' for i in range(1, 23)]]

In [40]:
# Quick check that hero ids are unique (i.e., no double picking)
# https://www.kaggle.com/kuzand/dota-2-winner-prediction-multilayer-nn-pytorch
np.all(df_full_features[[f'{i}hero_id' for i in range(1, 23)]].nunique(axis=1) == 22)

True

Hot encoding by getting dummies for each hero_id feature. 

In [42]:
for i in range(1,23):
        df_full_features = pd.get_dummies(df_full_features, columns = [f'{i}hero_id'])

### Scaling 

Standard Scaler, skipping hot encoding since trying out cat vs adaboost

In [47]:
df_full_features_scaled1 = df_full_features.copy()
df_full_features_scaled[df_full_features.columns.tolist()] = StandardScaler().fit_transform(df_full_features_scaled[df_full_features.columns.tolist()])  # alternatively use StandardScaler

In [45]:
df_full_features_scaled2 = df_full_features.copy()
df_full_features_scaled[df_full_features.columns.tolist()] = MinMaxScaler().fit_transform(df_full_features_scaled[df_full_features.columns.tolist()])  # alternatively use StandardScaler

In [48]:
df_full_features_scaled1.head()

1is_pick  1team  2is_pick  2team  3is_pick  3team  4is_pick  \
match_id                                                                  
5115031896         1      0         1      1         1      0         1   
4865407245         1      1         1      0         1      1         1   
4889341660         1      1         1      0         1      1         1   
5050335216         1      0         1      1         1      0         1   
4860032461         1      1         1      0         1      1         1   

            4team  5is_pick  5team  6is_pick  6team  7is_pick  7team  \
match_id                                                               
5115031896      1         1      0         1      1         1      0   
4865407245      0         1      1         1      0         1      1   
4889341660      0         1      1         1      0         1      1   
5050335216      1         1      0         1      1         1      0   
4860032461      0         1      1         1      0         1      1   

            8is_pick  8team  9is_pick  9team  10is_pick  10team  11is_pick  \
match_id                                                                     
5115031896         1      1         1      1          1       0          1   
4865407245         1      0         1      0          1       1          1   
4889341660         1      0         1      0          1       1          1   
5050335216         1      1         1      1          1       0          1   
4860032461         1      0         1      0          1       1          1   

            11team  12is_pick  12team  13is_pick  13team  14is_pick  14team  \
match_id                                                                      
5115031896       0          1       1          1       0          1       1   
4865407245       1          1       0          1       1          1       0   
4889341660       1          1       0          1       1          1       0   
5050335216       0          1       1          1       0          1       1   
4860032461       1          1       0          1       1          1       0   

            15is_pick  15team  16is_pick  16team  17is_pick  17team  \
match_id                                                              
5115031896          1       1          1       0          1       1   
4865407245          1       0          1       1          1       0   
4889341660          1       0          1       1          1       0   
5050335216          1       1          1       0          1       1   
4860032461          1       0          1       1          1       0   

            18is_pick  18team  19is_pick  19team  20is_pick  20team  \
match_id                                                              
5115031896          1       0          1       1          1       0   
4865407245          1       1          1       0          1       1   
4889341660          1       1          1       0          1       1   
5050335216          1       0          1       1          1       0   
4860032461          1       1          1       0          1       1   

            21is_pick  21team  22is_pick  22team  1hero_id_2  1hero_id_3  \
match_id                                                                   
5115031896          1       0          1       1           0           0   
4865407245          1       1          1       0           0           0   
4889341660          1       1          1       0           0           0   
5050335216          1       0          1       1           0           0   
4860032461          1       1          1       0           0           0   

            1hero_id_4  1hero_id_6  1hero_id_7  1hero_id_9  1hero_id_10  \
match_id                                                                  
5115031896           0           0           0           0            0   
4865407245           0           0           0           0            0   
4889341660           0           0           0     

In [49]:
df_full_features_scaled.shape

(1162, 2088)

## Set targets, features, and verification set 

Should the combined train sets still have hero variables? did I do this right?

In [23]:
# Encoded train/test features

X_train = train_features
#y_train = train_targets['radiant_win']
X_test = test_features
#y_test = test_targets['radiant_win']

In [22]:
#scaler = StandardScaler()
#scaler.fit(X_train)
#scaler.fit(X_test)
#X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)

#dpc_log_scaled = dpc_log.fit(X_scaled_train, y_train)

### Train Validation Split

Train Validation Split

For now, let's just try this out with the hero_ohe feature set

In [26]:
heroes_df_train = X_train[[f'{i}hero_id' for i in range(1, 23)]]
heroes_df_test = X_test[[f'{i}hero_id' for i in range(1, 23)]]

In [30]:
y_train = train_targets['radiant_win'].values
y_test = test_targets['radiant_win'].values
X_train = heroes_df_train
X_test = heroes_df_test

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.1, random_state = SEED) ## see top, SEED = 17

In [32]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((940, 22), (105, 22), (940,), (105,))

In [18]:
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [ ]:
from catboost import CatBoostClassifier
#We'll use full_df_mod without dummies and mark categorial vars
X_, X_valid_ctb, y_train_part_ctb, y_valid_ctb = train_test_split(full_df_mod[:idx_split], 
                                                                        y_train, test_size = 0.3, random_state=0) #fixing random_state
cat_vars = ['r_invar_herotype_sum', 'd_invar_herotype_sum'] + r_firstblood + d_firstblood #all the vars that we got dummies of

#Let it train for 200 iterations not to wait too long
ctb = CatBoostClassifier(iterations = 200, random_state=1, verbose=False, task_type='GPU', eval_metric='AUC', cat_features=cat_vars)

#We'll look at an online validation plot
ctb.fit(X_train_part_ctb, y_train_part_ctb.astype(float), eval_set=(X_valid_ctb, y_valid_ctb.astype(float)), plot=True)

ctb_ho_score =  roc_auc_score(y_valid_ctb.astype(float), ctb.predict_proba(X_valid_ctb)[:,1])
ctb_cv_score = cross_val_score(ctb, full_df_mod[:idx_split], y_train.astype(float), cv=cv, scoring = 'roc_auc') 